In [1]:
%install-location $cwd/swift-install
%install '.package(path: "$cwd/SchwiftyNotebook_batch_boi")' SchwiftyNotebook_batch_boi

Installing packages:
	.package(path: "/home/ubuntu/workspace/fast-ai-swift/SchwiftyNotebook_batch_boi")
		SchwiftyNotebook_batch_boi
With SwiftPM flags: []
Working in: /tmp/tmp3v2v66e_/swift-install
/home/ubuntu/swift/usr/bin/swift: /home/ubuntu/anaconda3/envs/swift-env/lib/libuuid.so.1: no version information available (required by /home/ubuntu/swift/usr/bin/swift)
/home/ubuntu/swift/usr/bin/swift: /home/ubuntu/anaconda3/envs/swift-env/lib/libuuid.so.1: no version information available (required by /home/ubuntu/swift/usr/bin/swift)
/home/ubuntu/swift/usr/bin/swift: /home/ubuntu/anaconda3/envs/swift-env/lib/libuuid.so.1: no version information available (required by /home/ubuntu/swift/usr/bin/swift)
/home/ubuntu/swift/usr/bin/swift: /home/ubuntu/anaconda3/envs/swift-env/lib/libuuid.so.1: no version information available (required by /home/ubuntu/swift/usr/bin/swift)
/home/ubuntu/swift/usr/bin/swift: /home/ubuntu/anaconda3/envs/swift-env/lib/libuuid.so.1: no version information availabl

In [2]:
//export
import Path
import TensorFlow
import SchwiftyNotebook_batch_boi

In [3]:
//export
public typealias SingleValueAndGradientChain<T>=(value: T, gradientChain: (T) -> T )
public typealias SingleInputDifferentiable<T> = (T) -> SingleValueAndGradientChain<T>

In [4]:
//export
public let identititySVGC: SingleInputDifferentiable<TensorFloat> = { x in (value: x, gradientChain: { y in y }) }

In [5]:
//export
public struct HyperParameter {
    let learningRate: Float
    
    init(learningRate: Float){
        self.learningRate = learningRate
    }
}

In [6]:
//export
public struct RuntimeError: Error {
    let message: String

    init(_ message: String) {
        self.message = message
    }

    public var localizedDescription: String {
        return message
    }
}

In [7]:
//export
public protocol ModelParameter  {
    
    func forwardPass(inputTensor: TensorFloat) -> (TensorFloat, ModelParameter)
    
    func apply(_ inputTensor: TensorFloat) -> TensorFloat 
    
    func backwardPass(ddx: TensorFloat, hyperParameters: HyperParameter) -> (TensorFloat, ModelParameter)
}

In [8]:
public struct YeetCannon: ModelParameter {
    public func backwardPass(ddx: TensorFloat, hyperParameters: HyperParameter) -> (TensorFloat, ModelParameter) {
        return (ddx, self)
    }
    
    public func apply(_ inputTensor: TensorFloat) -> TensorFloat {
        let (a, _) = forwardPass(inputTensor: inputTensor)
        return a
    } 
    
    public func forwardPass(inputTensor: TensorFloat) -> (TensorFloat, ModelParameter) {
        return (inputTensor, self)
    }
}

In [10]:
let yeetCannon: ModelParameter = YeetCannon()
let result = yeetCannon.apply(TensorFloat([Float(0.0)]))

In [11]:
//export
public typealias ThreeInputGradient = (TensorFloat) -> (TensorFloat, TensorFloat, TensorFloat)

public struct DenseLayer: ModelParameter {
    let activationFunction: SingleInputDifferentiable<TensorFloat>
    let weightParameter: TensorFloat
    let biasParameter: TensorFloat
    private let gradientChain: ThreeInputGradient

    private init(activationFunction: @escaping SingleInputDifferentiable<TensorFloat>,
                 weightParameter: TensorFloat,
                 biasParameter: TensorFloat,
                 gradientChain: @escaping ThreeInputGradient) {
        self.activationFunction = activationFunction
        self.weightParameter = weightParameter
        self.biasParameter = biasParameter
        self.gradientChain = gradientChain
    }

    private init(denseLayer: DenseLayer,
                 gradientChain: @escaping ThreeInputGradient) {
        self.activationFunction = denseLayer.activationFunction
        self.weightParameter = denseLayer.weightParameter
        self.biasParameter = denseLayer.biasParameter
        self.gradientChain = gradientChain
    }

    init(inputSize: Int,
         outputSize: Int,
         activationFunction: @escaping SingleInputDifferentiable<TensorFloat> = identititySVGC){
        self.activationFunction = activationFunction
        self.weightParameter = TensorFloat(kaimingUniform: TensorShape([inputSize, outputSize]))
        self.biasParameter = TensorFloat(zeros: [outputSize])
        self.gradientChain = { (x: TensorFloat)  in (TensorFloat([1]), TensorFloat([1]), TensorFloat([1]))}
    }

    public func forwardPass(inputTensor: TensorFloat) -> (TensorFloat, ModelParameter) {
        let (parameterOutput, parameterGradientChain) =
          linearCombinationAndGradient(inputTensor: inputTensor,
                                       weightParameter: self.weightParameter,
                                       biasParameter: self.biasParameter)
        let (activations, activationsGradientChain) = activationFunction(parameterOutput)

        return (activations, DenseLayer(denseLayer: self,
                                       gradientChain: {ddx in parameterGradientChain(activationsGradientChain(ddx))}))
    }

    public func apply(_ inputTensor: TensorFloat) -> TensorFloat {
        let parameterOutput =
          linearCombination(inputs: inputTensor,
                                       weights: self.weightParameter,
                                       bias: self.biasParameter)
        let (activations, activationsGradientChain) = activationFunction(parameterOutput)
        return activations
    }

    public func backwardPass(ddx: TensorFloat, hyperParameters: HyperParameter) -> (TensorFloat, ModelParameter) {
        let (ddxInput, ddxParameter, ddxBias) = self.gradientChain(ddx)
        return (ddxInput, DenseLayer(activationFunction: activationFunction,
                                    weightParameter: self.weightParameter - hyperParameters.learningRate * ddxParameter,
                                    biasParameter: self.biasParameter - hyperParameters.learningRate * ddxBias,
                                    gradientChain: self.gradientChain))
    }
}

In [12]:
let usedCarDataBunch = fetchUsedCarDataBunch(validationSize: 0.001, batchSize:1024)

In [13]:
let firstBatch = usedCarDataBunch.trainingDataset.first!
let numberFeatures = firstBatch.features.shape[1]
print(firstBatch.features.max(alongAxes: 0))
numberFeatures

[[0.82608694,        1.0,        1.0,        1.0,        1.0,        1.0,  0.9990104, 0.09999999]]


8


In [14]:
let carPriceLayer = DenseLayer(inputSize: numberFeatures, 
                               outputSize: 1, 
                               activationFunction: reLUAndGradient)

In [16]:
let preds = carPriceLayer.apply(firstBatch.features)

In [17]:
(preds.shape, firstBatch.labels.shape)

▿ 2 elements
  ▿ .0 : [373, 1]
    ▿ dimensions : 2 elements
      - 0 : 373
      - 1 : 1
  ▿ .1 : [373]
    ▿ dimensions : 1 element
      - 0 : 373


In [18]:
let (lossyBoi, gradientChain) = meanSquaredErrorAndGradient(yHat: preds, 
                                                            y: firstBatch.labels.expandingShape(at: 1))

In [19]:
lossyBoi

1.6948068e+12


In [20]:
var currentDenseLayer: ModelParameter = carPriceLayer
let trialsHyperParameter = HyperParameter(learningRate: 1e-2)
timeTrials(trials: 2) { 
    let (alexYHat, alexLayer) = currentDenseLayer.forwardPass(inputTensor: firstBatch.features)
    let (loss, lossGradientSupplier) = meanSquaredErrorAndGradient(yHat: alexYHat, 
                                                                   y:firstBatch.labels.expandingShape(at: 1))
    let (ddxAlexLayer, enhancedLayer) = alexLayer.backwardPass(ddx: lossGradientSupplier(), 
                                                               hyperParameters: trialsHyperParameter)
    currentDenseLayer = enhancedLayer
}

average: 2.211219 ms,   min: 2.162919 ms,   max: 2.259519 ms


In [22]:
let (nextLoss, _) = meanSquaredErrorAndGradient(yHat: currentDenseLayer.apply(firstBatch.features), 
                                                            y: firstBatch.labels.expandingShape(at: 1))
nextLoss

1.5106538e+12


In [27]:
var currentDenseLayer: ModelParameter = carPriceLayer
let trialsHyperParameter = HyperParameter(learningRate: 1e-2)
let firstValidation = usedCarDataBunch.validationDataset.first!
let (trainingLoss, _) = meanSquaredErrorAndGradient(yHat: currentDenseLayer.apply(firstValidation.features), 
                                                            y: firstValidation.labels.expandingShape(at: 1))
print("First Loss: \(trainingLoss)")
withTime {
    timeTrials(trials: 200) { 
        for batch in usedCarDataBunch.trainingDataset {
            let (alexYHat, alexLayer) = currentDenseLayer.forwardPass(inputTensor: batch.features)
            let (loss, lossGradientSupplier) = meanSquaredErrorAndGradient(yHat: alexYHat, 
                                                                           y:batch.labels.expandingShape(at: 1))
            let (ddxAlexLayer, enhancedLayer) = alexLayer.backwardPass(ddx: lossGradientSupplier(), 
                                                                       hyperParameters: trialsHyperParameter)
            currentDenseLayer = enhancedLayer   
        }
    }
}
let (postLoss, _) = meanSquaredErrorAndGradient(yHat: currentDenseLayer.apply(firstValidation.features), 
                                                            y: firstValidation.labels.expandingShape(at: 1))
print("Post Loss: \(postLoss)")
assert(trainingLoss > postLoss)

First Loss: 9.61809e+11
average: 99.59654681 ms,   min: 92.672819 ms,   max: 106.44483 ms
elapsed time: 20.02681291 s
Post Loss: 3.217901e+11


In [29]:
let predictedPrices = currentDenseLayer.apply(firstValidation.features)
let error = firstValidation.labels.expandingShape(at: 1) - predictedPrices

In [30]:
(error.mean(), error.standardDeviation())

▿ 2 elements
  - .0 : -6264.8853
  - .1 : 567230.94


In [31]:
(predictedPrices[6],  firstValidation.labels[6])

▿ 2 elements
  - .0 : [1142368.2]
  - .1 : 450000.0


In [35]:
var currentDenseLayer: ModelParameter = carPriceLayer
let trialsHyperParameter = HyperParameter(learningRate: 1e-1)
withTime {
    var trial = 0
    timeTrials(trials: 200) { 
        var accumulatedTrainingLoss = TensorFloat([0])
        var trainingItems = TensorFloat([0])
        for batch in usedCarDataBunch.trainingDataset {
            let (alexYHat, alexLayer) = currentDenseLayer.forwardPass(inputTensor: batch.features)
            
            let (loss, lossGradientSupplier) = meanSquaredErrorAndGradient(yHat: alexYHat, 
                                                                           y:batch.labels.expandingShape(at: 1))
            accumulatedTrainingLoss += loss
            trainingItems += 1
            
            let (ddxAlexLayer, enhancedLayer) = alexLayer.backwardPass(ddx: lossGradientSupplier(), 
                                                                       hyperParameters: trialsHyperParameter)
            currentDenseLayer = enhancedLayer   
        }
        
        
        var accumulatedValidationLoss = TensorFloat([0])
        var validationItems = TensorFloat([0])
        for validationBatch in usedCarDataBunch.validationDataset {
            let (validationLoss, _) = meanSquaredErrorAndGradient(yHat: currentDenseLayer.apply(validationBatch.features), 
                                                                  y: validationBatch.labels.expandingShape(at: 1))
            
            accumulatedValidationLoss += validationLoss
            validationItems += 1
        }
        
        if(trial % 10 == 0) {
            print("Training Loss: \(accumulatedTrainingLoss/trainingItems)")
            print("Validation Loss: \(accumulatedValidationLoss/validationItems)")   
        }
        trial += 1
    }
}

Training Loss: [2.3749777e+12]
Validation Loss: [4.642062e+11]
Training Loss: [2.21292e+12]
Validation Loss: [3.41256e+11]
Training Loss: [2.1985971e+12]
Validation Loss: [3.2018547e+11]
Training Loss: [2.1932633e+12]
Validation Loss: [3.1377257e+11]
Training Loss: [2.1901794e+12]
Validation Loss: [3.099206e+11]
Training Loss: [2.1879527e+12]
Validation Loss: [3.066774e+11]
Training Loss: [2.1862115e+12]
Validation Loss: [3.0376975e+11]
Training Loss: [2.1849094e+12]
Validation Loss: [3.01166e+11]
Training Loss: [2.1839024e+12]
Validation Loss: [2.987915e+11]
Training Loss: [2.1830916e+12]
Validation Loss: [2.965277e+11]
Training Loss: [2.1823934e+12]
Validation Loss: [2.9404332e+11]
Training Loss: [2.1814591e+12]
Validation Loss: [2.913005e+11]
Training Loss: [2.1805977e+12]
Validation Loss: [2.8902113e+11]
Training Loss: [2.1798663e+12]
Validation Loss: [2.8716197e+11]
Training Loss: [2.1792543e+12]
Validation Loss: [2.855083e+11]
Training Loss: [2.1787401e+12]
Validation Loss: [2.83

In [36]:
exportSchwiftyNotebook(notebookName: "Layer_Refined")

success
